只需要改end_date 和 tickers

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", "2892.TW", 
               "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", "0050.TW", "5880.TW", 
               "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", "00885.TW", "00895.TW", "00662.TW", 
               "00888.TW", "00646.TW", "00903.TW", "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", 
               "0051.TW", "00851.TW", "006203.TW", "00660.TW"]
    return tickers

def analyze_stock(ticker, end_date):
    # Download Historical Stock Data
    data = yf.download(ticker, start='2020-01-01', end=end_date)
    if data.empty:
        return None
    
    data.to_csv(f'{ticker}_stock_data.csv')
    
    # Technical Analysis
    df = pd.read_csv(f'{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()  # Remove leading/trailing whitespace
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Calculate daily returns
    df['Return'] = df['Close'].pct_change()

    # Calculate monthly returns
    monthly_return = df['Return'].resample('ME').mean()

    # Calculate technical indicators
    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)

    # Prepare data for Prophet model
    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})

    # Train Prophet model
    model = Prophet()
    model.fit(prophet_df)

    # Predict next 30 days
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    # Calculate win rate over forecast period
    forecast['Trend'] = forecast['yhat'].diff()
    up_days = forecast[forecast['Trend'] > 0].shape[0]
    total_days = forecast.shape[0]
    win_rate = up_days / total_days * 100

    # Calculate returns and win rates
    end_date = pd.to_datetime(end_date)
    one_month_return = df.loc[end_date - pd.DateOffset(months=1):end_date, 'Return'].mean() * 100
    six_months_return = df.loc[end_date - pd.DateOffset(months=6):end_date, 'Return'].mean() * 100
    one_year_return = df.loc[end_date - pd.DateOffset(years=1):end_date, 'Return'].mean() * 100

    six_months_positive = (df.loc[end_date - pd.DateOffset(months=6):end_date, 'Return'] > 0).sum()
    six_months_total = df.loc[end_date - pd.DateOffset(months=6):end_date, 'Return'].count()
    six_months_win_rate = six_months_positive / six_months_total * 100

    one_year_positive = (df.loc[end_date - pd.DateOffset(years=1):end_date, 'Return'] > 0).sum()
    one_year_total = df.loc[end_date - pd.DateOffset(years=1):end_date, 'Return'].count()
    one_year_win_rate = one_year_positive / one_year_total * 100

    return {
        'Ticker': ticker,
        'One Month Return': one_month_return,
        'Six Months Return': six_months_return,
        'One Year Return': one_year_return,
        'One Month Win Rate': win_rate,
        'Six Months Win Rate': six_months_win_rate,
        'One Year Win Rate': one_year_win_rate
    }

# Automatically fetch stock tickers
stock_tickers = get_stock_tickers()

# Define the end date for analysis
end_date = '2024-07-15'  # 修改这里的日期为当前日期

results = []

for ticker in stock_tickers:
    result = analyze_stock(ticker, end_date)
    if result:
        results.append(result)

# Create DataFrame from results and sort by One Month Return
results_df = pd.DataFrame(results)
top_10_bullish = results_df.sort_values(by='One Month Return', ascending=False).head(10)
top_10_bearish = results_df.sort_values(by='One Month Return').head(10)

# Save results to CSV
top_10_bullish.to_csv("top_10_bullish_ETF.csv", index=False)
top_10_bearish.to_csv("top_10_bearish_ETF.csv", index=False)

print("Top 10 Bullish Stocks:")
print(top_10_bullish)

print("\nTop 10 Bearish Stocks:")
print(top_10_bearish)


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", "2892.TW", 
               "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", "0050.TW", "5880.TW", 
               "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", "00885.TW", "00895.TW", "00662.TW", 
               "00888.TW", "00646.TW", "00903.TW", "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", 
               "0051.TW", "00851.TW", "006203.TW", "00660.TW"]
    return tickers

def analyze_stock(ticker):
    # Download Historical Stock Data
    data = yf.download(ticker, start='2020-01-01', end='2024-07-15')
    if data.empty:
        return None, None, None, None, None
    
    data.to_csv(f'{ticker}_stock_data.csv')
    
    # Technical Analysis
    df = pd.read_csv(f'{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()  # Remove leading/trailing whitespace
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Calculate daily returns
    df['Return'] = df['Close'].pct_change()

    # Calculate win rates
    monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()
    six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'] > 0).mean() * 100
    one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'] > 0).mean() * 100

    # Calculate technical indicators
    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)

    # Prepare data for Prophet model
    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})

    # Train Prophet model
    model = Prophet()
    model.fit(prophet_df)

    # Predict next 30 days
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    # Calculate win rate over forecast period
    forecast['Trend'] = forecast['yhat'].diff()
    up_days = forecast[forecast['Trend'] > 0].shape[0]
    total_days = forecast.shape[0]
    win_rate = up_days / total_days * 100

    # Calculate one month return
    one_month_return = df.loc[df.index[-1] - pd.DateOffset(months=1):, 'Return'].mean() * 100

    return one_month_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate

# Automatically fetch stock tickers
stock_tickers = get_stock_tickers()

results = []

for ticker in stock_tickers:  # Analyzing the tickers for demonstration purposes
    one_month_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate = analyze_stock(ticker)
    if one_month_return is not None:
        results.append({'Ticker': ticker, 'One Month Return': one_month_return, 'Monthly Win Rate': monthly_win_rate, 'Weekly Win Rate': weekly_win_rate, 'Six Months Win Rate': six_months_win_rate, 'One Year Win Rate': one_year_win_rate})

# Create DataFrame from results and sort by One Month Return
results_df = pd.DataFrame(results)
results_df['One Month Return'] = results_df['One Month Return']  # Already in percentage
results_df['Monthly Win Rate'] = results_df['Monthly Win Rate']  # Already in percentage
results_df['Weekly Win Rate'] = results_df['Weekly Win Rate']  # Already in percentage
results_df['Six Months Win Rate'] = results_df['Six Months Win Rate']  # Already in percentage
results_df['One Year Win Rate'] = results_df['One Year Win Rate']  # Already in percentage

top_10_stocks = results_df.sort_values(by='One Month Return', ascending=False).head(10)
bottom_10_stocks = results_df.sort_values(by='One Month Return', ascending=True).head(10)

# Save results to CSV
top_10_stocks.to_csv("top_10_bullish_stocks_etf.csv", index=False)
bottom_10_stocks.to_csv("bottom_10_bearish_stocks_etf.csv", index=False)

print(top_10_stocks)
print(bottom_10_stocks)


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", "2892.TW", 
               "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", "0050.TW", "5880.TW", 
               "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", "00885.TW", "00895.TW", "00662.TW", 
               "00888.TW", "00646.TW", "00903.TW", "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", 
               "0051.TW", "00851.TW", "006203.TW", "00660.TW"]
    return tickers

def analyze_stock(ticker):
    # Download Historical Stock Data
    data = yf.download(ticker, start='2020-01-01', end='2024-07-15')
    if data.empty:
        return None, None, None, None, None, None, None
    
    data.to_csv(f'{ticker}_stock_data.csv')
    
    # Technical Analysis
    df = pd.read_csv(f'{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()  # Remove leading/trailing whitespace
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Calculate daily returns
    df['Return'] = df['Close'].pct_change()

    # Calculate win rates
    monthly_win_rate = (df['Return'].resample('ME').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()
    six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'] > 0).mean() * 100
    one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'] > 0).mean() * 100

    # Calculate technical indicators
    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)

    # Prepare data for Prophet model
    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})

    # Train Prophet model
    model = Prophet()
    model.fit(prophet_df)

    # Predict next 30 days
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    # Calculate win rate over forecast period
    forecast['Trend'] = forecast['yhat'].diff()
    up_days = forecast[forecast['Trend'] > 0].shape[0]
    total_days = forecast.shape[0]
    win_rate = up_days / total_days * 100

    # Calculate returns
    one_month_return = df.loc[df.index[-1] - pd.DateOffset(months=1):, 'Return'].mean() * 100
    six_months_return = df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].mean() * 100
    one_year_return = df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].mean() * 100

    return one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate

# Automatically fetch stock tickers
stock_tickers = get_stock_tickers()

results = []

for ticker in stock_tickers:  # Analyzing the tickers for demonstration purposes
    one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate = analyze_stock(ticker)
    if one_month_return is not None:
        results.append({
            'Ticker': ticker, 
            'One Month Return': f'{one_month_return:.2f}%', 
            'Six Months Return': f'{six_months_return:.2f}%', 
            'One Year Return': f'{one_year_return:.2f}%', 
            'Monthly Win Rate': f'{monthly_win_rate:.2f}%', 
            'Weekly Win Rate': f'{weekly_win_rate:.2f}%', 
            'Six Months Win Rate': f'{six_months_win_rate:.2f}%', 
            'One Year Win Rate': f'{one_year_win_rate:.2f}%'
        })

# Create DataFrame from results and sort by One Month Return
results_df = pd.DataFrame(results)

top_10_stocks = results_df.sort_values(by='One Month Return', ascending=False).head(10)
bottom_10_stocks = results_df.sort_values(by='One Month Return', ascending=True).head(10)

# Save results to CSV
top_10_stocks.to_csv("top_10_bullish_stocks_etf.csv", index=False)
bottom_10_stocks.to_csv("bottom_10_bearish_stocks_etf.csv", index=False)

print(top_10_stocks)
print(bottom_10_stocks)


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt
import os

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", "2892.TW", 
               "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", "0050.TW", "5880.TW", 
               "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", "00885.TW", "00895.TW", "00662.TW", 
               "00888.TW", "00646.TW", "00903.TW", "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", 
               "0051.TW", "00851.TW", "006203.TW", "00660.TW"]
    return tickers

def analyze_stock(ticker):
    base_folder = f'results/{ticker}'
    os.makedirs(base_folder, exist_ok=True)
    
    # Download Historical Stock Data
    data = yf.download(ticker, start='2020-01-01', end='2024-07-15')
    if data.empty:
        return None, None, None, None, None, None, None
    
    data.to_csv(f'{base_folder}/{ticker}_stock_data.csv')
    
    # Technical Analysis
    df = pd.read_csv(f'{base_folder}/{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()  # Remove leading/trailing whitespace
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Calculate daily returns
    df['Return'] = df['Close'].pct_change()

    # Calculate win rates
    monthly_win_rate = (df['Return'].resample('ME').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()
    six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'] > 0).mean() * 100
    one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'] > 0).mean() * 100

    # Calculate technical indicators
    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)

    # Plot technical indicators
    plt.figure(figsize=(14, 7))
    plt.plot(df['Close'], label='Close Price')
    plt.plot(df['MA10'], label='MA10')
    plt.plot(df['MA20'], label='MA20')
    plt.title(f'{ticker} - Close Price and Moving Averages')
    plt.legend()
    plt.savefig(f'{base_folder}/{ticker}_technical_analysis.png')
    plt.close()
    
    plt.figure(figsize=(14, 7))
    plt.plot(df['RSI'], label='RSI')
    plt.title(f'{ticker} - RSI')
    plt.legend()
    plt.savefig(f'{base_folder}/{ticker}_RSI.png')
    plt.close()

    # Prepare data for Prophet model
    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})

    # Train Prophet model
    model = Prophet()
    model.fit(prophet_df)

    # Predict next 30 days
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    # Plot Prophet forecast
    fig = model.plot(forecast)
    plt.title(f'{ticker} - Prophet Forecast')
    plt.savefig(f'{base_folder}/{ticker}_prophet_forecast.png')
    plt.close()

    # Calculate win rate over forecast period
    forecast['Trend'] = forecast['yhat'].diff()
    up_days = forecast[forecast['Trend'] > 0].shape[0]
    total_days = forecast.shape[0]
    win_rate = up_days / total_days * 100

    # Calculate returns
    one_month_return = df.loc[df.index[-1] - pd.DateOffset(months=1):, 'Return'].mean() * 100
    six_months_return = df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].mean() * 100
    one_year_return = df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].mean() * 100

    return one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate

# Automatically fetch stock tickers
stock_tickers = get_stock_tickers()

results = []

for ticker in stock_tickers:  # Analyzing the tickers for demonstration purposes
    one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate = analyze_stock(ticker)
    if one_month_return is not None:
        results.append({
            'Ticker': ticker, 
            'One Month Return': f'{one_month_return:.2f}%', 
            'Six Months Return': f'{six_months_return:.2f}%', 
            'One Year Return': f'{one_year_return:.2f}%', 
            'Monthly Win Rate': f'{monthly_win_rate:.2f}%', 
            'Weekly Win Rate': f'{weekly_win_rate:.2f}%', 
            'Six Months Win Rate': f'{six_months_win_rate:.2f}%', 
            'One Year Win Rate': f'{one_year_win_rate:.2f}%'
        })

# Create DataFrame from results and sort by One Month Return
results_df = pd.DataFrame(results)

top_10_stocks = results_df.sort_values(by='One Month Return', ascending=False).head(10)
bottom_10_stocks = results_df.sort_values(by='One Month Return', ascending=True).head(10)

# Save results to separate folders
os.makedirs('results_ETF/bullish_stocks', exist_ok=True)
os.makedirs('results_ETF/bearish_stocks', exist_ok=True)

top_10_stocks.to_csv("results_ETF/bullish_stocks/top_10_bullish_stocks.csv", index=False)
bottom_10_stocks.to_csv("results_ETF/bearish_stocks/bottom_10_bearish_stocks.csv", index=False)

print(top_10_stocks)
print(bottom_10_stocks)


# 加入法人買賣和財經新聞

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt
import os

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", 
               "2892.TW", "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", 
               "0050.TW", "5880.TW", "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", 
               "00885.TW", "00895.TW", "00662.TW", "00888.TW", "00646.TW", "00903.TW", 
               "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", "0051.TW", "00851.TW", 
               "006203.TW", "00660.TW"]
    return tickers

def analyze_stock(ticker):
    data = yf.download(ticker, start='2020-01-01', end='2024-07-15')
    if data.empty:
        return None
    
    data.to_csv(f'results/{ticker}/{ticker}_stock_data.csv')
    
    df = pd.read_csv(f'results/{ticker}/{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['Return'] = df['Close'].pct_change()
    monthly_return = df['Return'].resample('M').mean()
    weekly_return = df['Return'].resample('W').mean()
    
    monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()

    try:
        six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'] > 0).mean() * 100
        one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'] > 0).mean() * 100
    except:
        six_months_win_rate = None
        one_year_win_rate = None

    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    
    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})
    model = Prophet()
    model.fit(prophet_df)
    
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    
    fig = model.plot(forecast)
    fig.savefig(f'results/{ticker}/{ticker}_prophet_forecast.png')
    plt.close(fig)
    
    df[['Close', 'MA10', 'MA20']].plot(title='Stock Prices and Moving Averages')
    plt.savefig(f'results/{ticker}/{ticker}_moving_averages.png')
    plt.close()
    
    df['RSI'].plot(title='Relative Strength Index (RSI)')
    plt.savefig(f'results/{ticker}/{ticker}_RSI.png')
    plt.close()
    
    one_month_return = df['Return'].last('1M').mean() * 100
    six_months_return = df['Return'].last('6M').mean() * 100
    one_year_return = df['Return'].last('1Y').mean() * 100
    
    daily_win_rate = (df['Return'] > 0).mean() * 100
    
    return one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, daily_win_rate

def save_results_to_csv(results, filename):
    results_df = pd.DataFrame(results)
    results_df.to_csv(filename, index=False)

if __name__ == "__main__":
    stock_tickers = get_stock_tickers()
    results = []
    
    if not os.path.exists('results'):
        os.mkdir('results')
    
    for ticker in stock_tickers:
        os.makedirs(f'results/{ticker}', exist_ok=True)
        result = analyze_stock(ticker)
        if result is not None:
            one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, daily_win_rate = result
            results.append({
                'Ticker': ticker, 
                'One Month Return': f'{one_month_return:.2f}%', 
                'Six Months Return': f'{six_months_return:.2f}%', 
                'One Year Return': f'{one_year_return:.2f}%', 
                'Monthly Win Rate': f'{monthly_win_rate:.2f}%', 
                'Weekly Win Rate': f'{weekly_win_rate:.2f}%', 
                'Six Months Win Rate': f'{six_months_win_rate:.2f}%' if six_months_win_rate is not None else 'N/A', 
                'One Year Win Rate': f'{one_year_win_rate:.2f}%' if one_year_win_rate is not None else 'N/A', 
                'Daily Win Rate': f'{daily_win_rate:.2f}%'
            })
    
    save_results_to_csv(results, 'results/ETF_performance_report.csv')
    print("ETF performance report saved.")


[*********************100%%**********************]  1 of 1 completed
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/1929668904.py:31: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_return = df['Return'].resample('M').mean()
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/1929668904.py:34: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
10:11:38 - cmdstanpy - INFO - Chain [1] start processing
10:11:38 - cmdstanpy - INFO - Chain [1] done processing
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` o

ETF performance report saved.


/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/1929668904.py:67: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  one_month_return = df['Return'].last('1M').mean() * 100
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/1929668904.py:67: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  one_month_return = df['Return'].last('1M').mean() * 100
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/1929668904.py:68: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  six_months_return = df['Return'].last('6M').mean() * 100
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/1929668904.py:68: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  six_months_return = df['Return'].last

## MACD（移動平均收斂/發散指標）
## 布林帶（Bollinger Bands）
## ATR（真實波幅均值）
## ADX（平均方向指標）


In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt
import os

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", 
               "2892.TW", "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", 
               "0050.TW", "5880.TW", "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", 
               "00885.TW", "00895.TW", "00662.TW", "00888.TW", "00646.TW", "00903.TW", 
               "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", "0051.TW", "00851.TW", 
               "006203.TW", "00660.TW"]
    return tickers

def analyze_stock(ticker):
    data = yf.download(ticker, start='2020-01-01', end='2024-07-15')
    if data.empty:
        return None
    
    if not os.path.exists(f'results_ETF_0715/{ticker}'):
        os.makedirs(f'results_ETF_0715/{ticker}')
    
    data.to_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    
    df = pd.read_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['Return'] = df['Close'].pct_change()
    monthly_return = df['Return'].resample('M').mean()
    weekly_return = df['Return'].resample('W').mean()
    daily_return = df['Return']

    monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()

    try:
        six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'] > 0).mean() * 100
        one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'] > 0).mean() * 100
    except:
        six_months_win_rate = None
        one_year_win_rate = None

    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = talib.BBANDS(df['Close'], timeperiod=20)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

    df[['Close', 'MA10', 'MA20', 'BB_Upper', 'BB_Middle', 'BB_Lower']].plot(title='Stock Prices and Indicators')
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_indicators.png')
    plt.close()

    df['RSI'].plot(title='Relative Strength Index (RSI)')
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_RSI.png')
    plt.close()

    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})
    model = Prophet()
    model.fit(prophet_df)
    
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    
    fig = model.plot(forecast)
    fig.savefig(f'results_ETF_0715/{ticker}/{ticker}_prophet_forecast.png')
    plt.close(fig)
    
    one_month_return = df['Return'].last('1M').mean() * 100
    six_months_return = df['Return'].last('6M').mean() * 100
    one_year_return = df['Return'].last('1Y').mean() * 100
    
    daily_win_rate = (df['Return'] > 0).mean() * 100

    return one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, daily_win_rate, weekly_return.mean() * 100, daily_return.mean() * 100

def save_results_to_csv(results, filename):
    results_df = pd.DataFrame(results)
    results_df.to_csv(filename, index=False)

if __name__ == "__main__":
    stock_tickers = get_stock_tickers()
    results = []
    
    for ticker in stock_tickers:
        result = analyze_stock(ticker)
        if result is not None:
            one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, daily_win_rate, weekly_return, daily_return = result
            results.append({
                'Ticker': ticker, 
                'One Month Return': f'{one_month_return:.2f}%', 
                'Six Months Return': f'{six_months_return:.2f}%', 
                'One Year Return': f'{one_year_return:.2f}%', 
                'Monthly Win Rate': f'{monthly_win_rate:.2f}%', 
                'Weekly Win Rate': f'{weekly_win_rate:.2f}%', 
                'Six Months Win Rate': f'{six_months_win_rate:.2f}%' if six_months_win_rate is not None else 'N/A', 
                'One Year Win Rate': f'{one_year_win_rate:.2f}%' if one_year_win_rate is not None else 'N/A', 
                'Daily Win Rate': f'{daily_win_rate:.2f}%',
                'Weekly Return': f'{weekly_return:.2f}%',
                'Daily Return': f'{daily_return:.2f}%'
            })
    
    save_results_to_csv(results, 'results_ETF_0715/ETF_performance_report.csv')
    print("ETF performance report saved.")


[*********************100%%**********************]  1 of 1 completed


/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/998035183.py:34: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_return = df['Return'].resample('M').mean()
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/998035183.py:38: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
10:42:45 - cmdstanpy - INFO - Chain [1] start processing
10:42:45 - cmdstanpy - INFO - Chain [1] done processing
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()
/opt/anaconda3/e

ETF performance report saved.


/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/998035183.py:75: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  one_month_return = df['Return'].last('1M').mean() * 100
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/998035183.py:75: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  one_month_return = df['Return'].last('1M').mean() * 100
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/998035183.py:76: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  six_months_return = df['Return'].last('6M').mean() * 100
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_23960/998035183.py:76: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  six_months_return = df['Return'].last('6M

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt
import os
from yahoo_fin import stock_info as si

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", 
               "2892.TW", "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", 
               "0050.TW", "5880.TW", "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", 
               "00885.TW", "00895.TW", "00662.TW", "00888.TW", "00646.TW", "00903.TW", 
               "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", "0051.TW", "00851.TW", 
               "006203.TW", "00660.TW"]
    return tickers

def analyze_stock(ticker):
    data = yf.download(ticker, start='2020-01-01', end='2024-07-15')
    if data.empty:
        return None
    
    if not os.path.exists(f'results_ETF_0715/{ticker}'):
        os.makedirs(f'results_ETF_0715/{ticker}')
    
    data.to_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    
    df = pd.read_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['Return'] = df['Close'].pct_change()
    monthly_return = df['Return'].resample('M').mean()
    weekly_return = df['Return'].resample('W').mean()
    daily_return = df['Return']

    monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()
    daily_win_rate = (df['Return'] > 0).mean() * 100

    try:
        six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'] > 0).mean() * 100
        one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'] > 0).mean() * 100
    except:
        six_months_win_rate = None
        one_year_win_rate = None

    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = talib.BBANDS(df['Close'], timeperiod=20)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

    df[['Close', 'MA10', 'MA20', 'BB_Upper', 'BB_Middle', 'BB_Lower']].plot(title='Stock Prices and Indicators')
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_indicators.png')
    plt.close()

    df['RSI'].plot(title='Relative Strength Index (RSI)')
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_RSI.png')
    plt.close()

    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})
    model = Prophet()
    model.fit(prophet_df)
    
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    
    fig = model.plot(forecast)
    fig.savefig(f'results_ETF_0715/{ticker}/{ticker}_prophet_forecast.png')
    plt.close(fig)

    one_month_return = df['Return'].last('1M').mean() * 100
    six_months_return = df['Return'].last('6M').mean() * 100
    one_year_return = df['Return'].last('1Y').mean() * 100
    
    # 爬取新闻数据
    try:
        news = get_news(ticker)
        news.to_csv(f'results_ETF_0715/{ticker}/{ticker}_news.csv', index=False)
    except Exception as e:
        print(f"Failed to retrieve news for {ticker}: {e}")
    
    return one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, daily_win_rate, weekly_return.mean() * 100, daily_return.mean() * 100

def get_news(ticker):
    news_table = si.get_news(ticker)
    news_list = []
    for news in news_table:
        news_list.append({
            'Date': news['date'],
            'Title': news['title'],
            'Link': news['link']
        })
    news_df = pd.DataFrame(news_list[:20])  # 取前20条新闻
    return news_df

def save_results_to_csv(results, filename):
    results_df = pd.DataFrame(results)
    results_df.to_csv(filename, index=False)

if __name__ == "__main__":
    stock_tickers = get_stock_tickers()
    results = []
    
    for ticker in stock_tickers:
        result = analyze_stock(ticker)
        if result is not None:
            one_month_return, six_months_return, one_year_return, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, daily_win_rate, weekly_return, daily_return = result
            results.append({
                'Ticker': ticker, 
                'Daily Return': f'{daily_return:.2f}%',
                'Weekly Return': f'{weekly_return:.2f}%', 
                'One Month Return': f'{one_month_return:.2f}%', 
                'Six Months Return': f'{six_months_return:.2f}%', 
                'One Year Return': f'{one_year_return:.2f}%', 
                'Daily Win Rate': f'{daily_win_rate:.2f}%',
                'Monthly Win Rate': f'{monthly_win_rate:.2f}%', 
                'Weekly Win Rate': f'{weekly_win_rate:.2f}%', 
                'Six Months Win Rate': f'{six_months_win_rate:.2f}%' if six_months_win_rate is not None else 'N/A', 
                'One Year Win Rate': f'{one_year_win_rate:.2f}%' if one_year_win_rate is not None else 'N/A', 
             })

    save_results_to_csv(results, 'results_ETF_0715/ETF_performance_report.csv')
    print("ETF performance report saved.")


In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt
import os
import requests
from bs4 import BeautifulSoup

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", 
               "2892.TW", "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", 
               "0050.TW", "5880.TW", "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", 
               "00885.TW", "00895.TW", "00662.TW", "00888.TW", "00646.TW", "00903.TW", 
               "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", "0051.TW", "00851.TW", 
               "006203.TW", "00660.TW"]
    return tickers

def fetch_news(ticker):
    url = f"https://tw.stock.yahoo.com/q/h?s={ticker}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_items = soup.find_all('li', class_='js-stream-content')

    news_list = []
    for item in news_items[:20]:  # Get top 20 news
        title = item.find('h3').text if item.find('h3') else 'No title'
        link = item.find('a')['href'] if item.find('a') else 'No link'
        news_list.append({'Title': title, 'Link': link})
    
    return news_list

def fetch_institutional_trading(ticker):
    # Simulated function to fetch institutional trading data
    # In real implementation, you would fetch this data from a reliable source
    dates = pd.date_range(end=pd.Timestamp('today'), periods=30)
    data = {
        'Date': dates,
        'Foreign_Investor_Buy': np.random.randint(100, 1000, size=30),
        'Foreign_Investor_Sell': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Buy': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Sell': np.random.randint(100, 1000, size=30),
        'Dealer_Buy': np.random.randint(100, 1000, size=30),
        'Dealer_Sell': np.random.randint(100, 1000, size=30)
    }
    df = pd.DataFrame(data)
    return df

def analyze_stock(ticker):
    data = yf.download(ticker, start='2020-01-01', end='2024-07-15')
    if data.empty:
        return None
    
    if not os.path.exists(f'results_ETF_0715/{ticker}'):
        os.makedirs(f'results_ETF_0715/{ticker}')
    
    data.to_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    
    df = pd.read_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['Return'] = df['Close'].pct_change()
    daily_return = df['Return'].mean() * 100
    weekly_return = df['Return'].resample('W').mean().mean() * 100
    one_month_return = df['Return'].last('1M').mean() * 100
    six_months_return = df['Return'].last('6M').mean() * 100
    one_year_return = df['Return'].last('1Y').mean() * 100

    daily_win_rate = (df['Return'].apply(lambda x: x > 0).mean()) * 100
    monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()
    six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].empty else None
    one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].empty else None

    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['Upper_BB'], df['Middle_BB'], df['Lower_BB'] = talib.BBANDS(df['Close'], timeperiod=20)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

    df[['Close', 'MA10', 'MA20', 'Upper_BB', 'Middle_BB', 'Lower_BB']].plot(title=f'{ticker} - Stock Prices and Indicators')
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_indicators.png')
    plt.close()

    df[['RSI', 'MACD', 'MACD_signal', 'ADX']].plot(title=f'{ticker} - Technical Indicators')
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_technical_indicators.png')
    plt.close()

    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})

    model = Prophet()
    model.fit(prophet_df)

    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    fig = model.plot(forecast)
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_prophet_forecast.png')
    plt.close()

    news_list = fetch_news(ticker)
    news_df = pd.DataFrame(news_list)
    news_df.to_csv(f'results_ETF_0715/{ticker}/{ticker}_news.csv', index=False)

    institutional_trading_df = fetch_institutional_trading(ticker)
    institutional_trading_df.plot(x='Date', title=f'{ticker} - Institutional Trading')
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_institutional_trading.png')
    plt.close()
    
    return daily_return, weekly_return, one_month_return, six_months_return, one_year_return, daily_win_rate, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, df

stock_tickers = get_stock_tickers()
results = []

for ticker in stock_tickers:
    result = analyze_stock(ticker)
    if result is not None:
        daily_return, weekly_return, one_month_return, six_months_return, one_year_return, daily_win_rate, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, df = result
        results.append({
            'Ticker': ticker, 
            'Daily Return': f'{daily_return:.2f}%', 
            'Weekly Return': f'{weekly_return:.2f}%', 
            'One Month Return': f'{one_month_return:.2f}%', 
            'Six Months Return': f'{six_months_return:.2f}%', 
            'One Year Return': f'{one_year_return:.2f}%', 
            'Daily Win Rate': f'{daily_win_rate:.2f}%', 
            'Monthly Win Rate': f'{monthly_win_rate:.2f}%', 
            'Weekly Win Rate': f'{weekly_win_rate:.2f}%', 
            'Six Months Win Rate': f'{six_months_win_rate:.2f}%' if six_months_win_rate is not None else 'N/A', 
            'One Year Win Rate': f'{one_year_win_rate:.2f}%' if one_year_win_rate is not None else 'N/A'
        })

# Create DataFrame from results and save to CSV
results_df = pd.DataFrame(results)
results_df = results_df[['Ticker', 'Daily Return', 'Weekly Return', 'One Month Return', 'Six Months Return', 'One Year Return', 
                         'Daily Win Rate', 'Monthly Win Rate', 'Weekly Win Rate', 'Six Months Win Rate', 'One Year Win Rate']]
results_df.to_csv("results_ETF_0715/results_summary.csv", index=False)


[*********************100%%**********************]  1 of 1 completed
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_27589/2447820441.py:68: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  one_month_return = df['Return'].last('1M').mean() * 100
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_27589/2447820441.py:68: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  one_month_return = df['Return'].last('1M').mean() * 100
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_27589/2447820441.py:69: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  six_months_return = df['Return'].last('6M').mean() * 100
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_27589/2447820441.py:69: FutureWarning: 'M' is deprecated and will be removed in a future vers

## 考慮各種指標、圖表呈現整合

In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
from bs4 import BeautifulSoup

# 设置Seaborn样式
sns.set(style='whitegrid')

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", 
               "2892.TW", "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", 
               "0050.TW", "5880.TW", "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", 
               "00885.TW", "00895.TW", "00662.TW", "00888.TW", "00646.TW", "00903.TW", 
               "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", "0051.TW", "00851.TW", 
               "006203.TW", "00660.TW"]
    return tickers

def fetch_news(ticker):
    url = f"https://tw.stock.yahoo.com/q/h?s={ticker}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_items = soup.find_all('li', class_='js-stream-content')

    news_list = []
    for item in news_items[:20]:  # Get top 20 news
        title = item.find('h3').text if item.find('h3') else 'No title'
        link = item.find('a')['href'] if item.find('a') else 'No link'
        news_list.append({'Title': title, 'Link': link})
    
    return news_list

def fetch_institutional_trading(ticker):
    # Simulated function to fetch institutional trading data
    # In real implementation, you would fetch this data from a reliable source
    dates = pd.date_range(end=pd.Timestamp('today'), periods=30)
    data = {
        'Date': dates,
        'Foreign_Investor_Buy': np.random.randint(100, 1000, size=30),
        'Foreign_Investor_Sell': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Buy': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Sell': np.random.randint(100, 1000, size=30),
        'Dealer_Buy': np.random.randint(100, 1000, size=30),
        'Dealer_Sell': np.random.randint(100, 1000, size=30)
    }
    df = pd.DataFrame(data)
    return df
def analyze_stock(ticker):
    data = yf.download(ticker, start='2020-01-01', end='2024-07-15')
    if data.empty:
        return None
    
    if not os.path.exists(f'results_ETF_0715/{ticker}'):
        os.makedirs(f'results_ETF_0715/{ticker}')
    
    data.to_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    
    df = pd.read_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['Return'] = df['Close'].pct_change()

    # Calculate technical indicators
    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['Upper_BB'], df['Middle_BB'], df['Lower_BB'] = talib.BBANDS(df['Close'], timeperiod=20)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

    # Plot technical indicators
    fig, ax = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

    df[['Close', 'MA10', 'MA20', 'Upper_BB', 'Middle_BB', 'Lower_BB']].plot(ax=ax[0], title=f'{ticker} - Stock Prices and Indicators')
    ax[0].set_ylabel('Price')
    
    df[['RSI']].plot(ax=ax[1], title='RSI')
    ax[1].set_ylabel('RSI')
    
    df[['MACD', 'MACD_signal']].plot(ax=ax[2], title='MACD')
    ax[2].fill_between(df.index, df['MACD_hist'], 0, color='gray', alpha=0.3)
    ax[2].set_ylabel('MACD')

    plt.tight_layout()
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_indicators.png')
    plt.close()

    # Plot institutional trading
    institutional_trading_df = fetch_institutional_trading(ticker)
    fig, axs = plt.subplots(3, 1, figsize=(14, 18), sharex=True)

    sns.lineplot(data=institutional_trading_df, x='Date', y='Foreign_Investor_Buy', label='Foreign Investor Buy', ax=axs[0])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Foreign_Investor_Sell', label='Foreign Investor Sell', ax=axs[0])
    axs[0].set_title(f'{ticker} - Foreign Investors')
    
    sns.lineplot(data=institutional_trading_df, x='Date', y='Investment_Trust_Buy', label='Investment Trust Buy', ax=axs[1])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Investment_Trust_Sell', label='Investment Trust Sell', ax=axs[1])
    axs[1].set_title(f'{ticker} - Investment Trusts')
    
    sns.lineplot(data=institutional_trading_df, x='Date', y='Dealer_Buy', label='Dealer Buy', ax=axs[2])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Dealer_Sell', label='Dealer Sell', ax=axs[2])
    axs[2].set_title(f'{ticker} - Dealers')
    
    plt.tight_layout()
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_institutional_trading.png')
    plt.close()

    # Prepare data for Prophet model
    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})

    # Train Prophet model
    model = Prophet()
    model.fit(prophet_df)

    # Predict next 30 days
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    # Plot forecast
    fig = model.plot(forecast)
    plt.title(f'{ticker} - Prophet Forecast')
    plt.tight_layout()
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_prophet_forecast.png')
    plt.close()

    # Fetch and save news
    news_list = fetch_news(ticker)
    news_df = pd.DataFrame(news_list)
    news_df.to_csv(f'results_ETF_0715/{ticker}/{ticker}_news.csv', index=False)

    # 计算每日、每周、每月、半年和年收益率及胜率
    daily_return = df['Return'].mean() * 100
    weekly_return = df['Return'].resample('W').mean().mean() * 100
    one_month_return = df.loc[df.index[-1] - pd.DateOffset(months=1):, 'Return'].mean() * 100
    six_months_return = df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].mean() * 100
    one_year_return = df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].mean() * 100

    daily_win_rate = (df['Return'].apply(lambda x: x > 0).mean()) * 100
    monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()
    six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].empty else None
    one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].empty else None

    return daily_return, weekly_return, one_month_return, six_months_return, one_year_return, daily_win_rate, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, df

stock_tickers = get_stock_tickers()
results = []

for ticker in stock_tickers:
    result = analyze_stock(ticker)
    if result is not None:
        daily_return, weekly_return, one_month_return, six_months_return, one_year_return, daily_win_rate, monthly_win_rate, weekly_win_rate, six_months_win_rate, one_year_win_rate, df = result
        results.append({
            'Ticker': ticker, 
            'Daily Return': f'{daily_return:.2f}%', 
            'Weekly Return': f'{weekly_return:.2f}%', 
            'One Month Return': f'{one_month_return:.2f}%', 
            'Six Months Return': f'{six_months_return:.2f}%', 
            'One Year Return': f'{one_year_return:.2f}%', 
            'Daily Win Rate': f'{daily_win_rate:.2f}%', 
            'Monthly Win Rate': f'{monthly_win_rate:.2f}%', 
            'Weekly Win Rate': f'{weekly_win_rate:.2f}%', 
            'Six Months Win Rate': f'{six_months_win_rate:.2f}%' if six_months_win_rate is not None else 'N/A', 
            'One Year Win Rate': f'{one_year_win_rate:.2f}%' if one_year_win_rate is not None else 'N/A'
        })

# Create DataFrame from results and save to CSV
results_df = pd.DataFrame(results)
results_df = results_df[['Ticker', 'Daily Return', 'Weekly Return', 'One Month Return', 'Six Months Return', 'One Year Return', 
                         'Daily Win Rate', 'Monthly Win Rate', 'Weekly Win Rate', 'Six Months Win Rate', 'One Year Win Rate']]
results_df.to_csv("results_ETF_0715/results_summary.csv", index=False)


[*********************100%%**********************]  1 of 1 completed
13:34:56 - cmdstanpy - INFO - Chain [1] start processing
13:34:56 - cmdstanpy - INFO - Chain [1] done processing
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:73: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  ax.plot(m.history['ds'].dt.to_pydatetime(), m.history['y'], 'k.',
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_27589/182338

# 加權平均勝率

In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
from bs4 import BeautifulSoup

# 设置Seaborn样式
sns.set(style='whitegrid')

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", 
               "2892.TW", "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", 
               "0050.TW", "5880.TW", "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", 
               "00885.TW", "00895.TW", "00662.TW", "00888.TW", "00646.TW", "00903.TW", 
               "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", "0051.TW", "00851.TW", 
               "006203.TW", "00660.TW"]
    return tickers

def fetch_news(ticker):
    url = f"https://tw.stock.yahoo.com/q/h?s={ticker}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_items = soup.find_all('li', class_='js-stream-content')

    news_list = []
    for item in news_items[:20]:  # Get top 20 news
        title = item.find('h3').text if item.find('h3') else 'No title'
        link = item.find('a')['href'] if item.find('a') else 'No link'
        news_list.append({'Title': title, 'Link': link})
    
    return news_list

def fetch_institutional_trading(ticker):
    # Simulated function to fetch institutional trading data
    # In real implementation, you would fetch this data from a reliable source
    dates = pd.date_range(end=pd.Timestamp('today'), periods=30)
    data = {
        'Date': dates,
        'Foreign_Investor_Buy': np.random.randint(100, 1000, size=30),
        'Foreign_Investor_Sell': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Buy': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Sell': np.random.randint(100, 1000, size=30),
        'Dealer_Buy': np.random.randint(100, 1000, size=30),
        'Dealer_Sell': np.random.randint(100, 1000, size=30)
    }
    df = pd.DataFrame(data)
    return df
def analyze_stock(ticker):
    data = yf.download(ticker, start='2020-01-01', end='2024-07-15')
    if data.empty:
        return None
    
    if not os.path.exists(f'results_ETF_0715/{ticker}'):
        os.makedirs(f'results_ETF_0715/{ticker}')
    
    data.to_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    
    df = pd.read_csv(f'results_ETF_0715/{ticker}/{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['Return'] = df['Close'].pct_change()

    # Calculate technical indicators
    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['Upper_BB'], df['Middle_BB'], df['Lower_BB'] = talib.BBANDS(df['Close'], timeperiod=20)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

    # Plot technical indicators
    fig, ax = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

    df[['Close', 'MA10', 'MA20', 'Upper_BB', 'Middle_BB', 'Lower_BB']].plot(ax=ax[0], title=f'{ticker} - Stock Prices and Indicators')
    ax[0].set_ylabel('Price')
    
    df[['RSI']].plot(ax=ax[1], title='RSI')
    ax[1].set_ylabel('RSI')
    
    df[['MACD', 'MACD_signal']].plot(ax=ax[2], title='MACD')
    ax[2].fill_between(df.index, df['MACD_hist'], 0, color='gray', alpha=0.3)
    ax[2].set_ylabel('MACD')

    plt.tight_layout()
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_indicators.png')
    plt.close()

    # Plot institutional trading
    institutional_trading_df = fetch_institutional_trading(ticker)
    fig, axs = plt.subplots(3, 1, figsize=(14, 18), sharex=True)

    sns.lineplot(data=institutional_trading_df, x='Date', y='Foreign_Investor_Buy', label='Foreign Investor Buy', ax=axs[0])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Foreign_Investor_Sell', label='Foreign Investor Sell', ax=axs[0])
    axs[0].set_title(f'{ticker} - Foreign Investors')
    
    sns.lineplot(data=institutional_trading_df, x='Date', y='Investment_Trust_Buy', label='Investment Trust Buy', ax=axs[1])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Investment_Trust_Sell', label='Investment Trust Sell', ax=axs[1])
    axs[1].set_title(f'{ticker} - Investment Trusts')
    
    sns.lineplot(data=institutional_trading_df, x='Date', y='Dealer_Buy', label='Dealer Buy', ax=axs[2])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Dealer_Sell', label='Dealer Sell', ax=axs[2])
    axs[2].set_title(f'{ticker} - Dealers')
    
    plt.tight_layout()
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_institutional_trading.png')
    plt.close()

    # Prepare data for Prophet model
    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})

    # Train Prophet model
    model = Prophet()
    model.fit(prophet_df)

    # Predict next 30 days
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    # Plot forecast
    fig = model.plot(forecast)
    plt.title(f'{ticker} - Prophet Forecast')
    plt.tight_layout()
    plt.savefig(f'results_ETF_0715/{ticker}/{ticker}_prophet_forecast.png')
    plt.close()

    # Fetch and save news
    news_list = fetch_news(ticker)
    news_df = pd.DataFrame(news_list)
    news_df.to_csv(f'results_ETF_0715/{ticker}/{ticker}_news.csv', index=False)

    # 计算每日、每周、每月、半年和年收益率及胜率
    daily_return = df['Return'].mean() * 100
    weekly_return = df['Return'].resample('W').mean().mean() * 100
    one_month_return = df.loc[df.index[-1] - pd.DateOffset(months=1):, 'Return'].mean() * 100
    six_months_return = df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].mean() * 100
    one_year_return = df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].mean() * 100

    daily_win_rate = (df['Return'].apply(lambda x: x > 0).mean()) * 100
    monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()
    six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].empty else None
    one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].empty else None

    # 根据给定条件计算额外的胜率
    additional_win_rate = 0
    if df['Close'].iloc[-1] > df['Middle_BB'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['Close'].iloc[-1] > df['MA20'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['Close'].iloc[-1] > df['MA10'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['Close'].iloc[-1] > df['Upper_BB'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['RSI'].iloc[-1] > 60:
        additional_win_rate += 5
    if df['MACD'].iloc[-1] > 2:
        additional_win_rate += 5

    # 判断Prophet预测趋势
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    if forecast['yhat'].iloc[-1] > forecast['yhat'].iloc[-2]:
        additional_win_rate += 5

    # 判断三大法人是否连续三天买超
    institutional_trading_df['Net_Buy'] = institutional_trading_df['Foreign_Investor_Buy'] - institutional_trading_df['Foreign_Investor_Sell'] + \
                                          institutional_trading_df['Investment_Trust_Buy'] - institutional_trading_df['Investment_Trust_Sell'] + \
                                          institutional_trading_df['Dealer_Buy'] - institutional_trading_df['Dealer_Sell']
    if (institutional_trading_df['Net_Buy'].tail(3) > 0).all():
        additional_win_rate += 5

    total_win_rate = daily_win_rate + weekly_win_rate + monthly_win_rate + (six_months_win_rate if six_months_win_rate is not None else 0) + (one_year_win_rate if one_year_win_rate is not None else 0) + additional_win_rate

    return {
        'ticker': ticker,
        'daily_return': daily_return,
        'weekly_return': weekly_return,
        'one_month_return': one_month_return,
        'six_months_return': six_months_return,
        'one_year_return': one_year_return,
        'daily_win_rate': daily_win_rate,
        'monthly_win_rate': monthly_win_rate,
        'weekly_win_rate': weekly_win_rate,
        'six_months_win_rate': six_months_win_rate,
        'one_year_win_rate': one_year_win_rate,
        'additional_win_rate': additional_win_rate,
        'total_win_rate': total_win_rate,
        'df': df
    }

stock_tickers = get_stock_tickers()
results = []

for ticker in stock_tickers:
    result = analyze_stock(ticker)
    if result is not None:
        results.append(result)

# 创建DataFrame保存结果并导出为CSV
results_df = pd.DataFrame(results)
results_df = results_df[['ticker', 'daily_return', 'weekly_return', 'one_month_return', 'six_months_return', 'one_year_return', 
                         'daily_win_rate', 'weekly_win_rate', 'monthly_win_rate', 'six_months_win_rate', 'one_year_win_rate', 
                         'additional_win_rate', 'total_win_rate']]
results_df.to_csv("results_ETF_0715/weighted_win_rates.csv", index=False)


[*********************100%%**********************]  1 of 1 completed
14:38:39 - cmdstanpy - INFO - Chain [1] start processing
14:38:39 - cmdstanpy - INFO - Chain [1] done processing
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:73: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  ax.plot(m.history['ds'].dt.to_pydatetime(), m.history['y'], 'k.',
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_27589/181733

# 7/15

In [22]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
from bs4 import BeautifulSoup

# 设置Seaborn样式
sns.set(style='whitegrid')

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", 
               "2892.TW", "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", 
               "0050.TW", "5880.TW", "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", 
               "00885.TW", "00895.TW", "00662.TW", "00888.TW", "00646.TW", "00903.TW", 
               "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", "0051.TW", "00851.TW", 
               "006203.TW", "00660.TW"]
    return tickers

def fetch_news(ticker):
    url = f"https://tw.stock.yahoo.com/q/h?s={ticker}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_items = soup.find_all('li', class_='js-stream-content')

    news_list = []
    for item in news_items[:20]:  # Get top 20 news
        title = item.find('h3').text if item.find('h3') else 'No title'
        link = item.find('a')['href'] if item.find('a') else 'No link'
        news_list.append({'Title': title, 'Link': link})
    
    return news_list

def fetch_institutional_trading(ticker):
    # Simulated function to fetch institutional trading data
    # In real implementation, you would fetch this data from a reliable source
    dates = pd.date_range(end=pd.Timestamp('today'), periods=30)
    data = {
        'Date': dates,
        'Foreign_Investor_Buy': np.random.randint(100, 1000, size=30),
        'Foreign_Investor_Sell': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Buy': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Sell': np.random.randint(100, 1000, size=30),
        'Dealer_Buy': np.random.randint(100, 1000, size=30),
        'Dealer_Sell': np.random.randint(100, 1000, size=30)
    }
    df = pd.DataFrame(data)
    return df
def analyze_stock(ticker):
    data = yf.download(ticker, start='2020-01-01', end='2024-07-16')
    if data.empty:
        return None
    
    if not os.path.exists(f'results_ETF_0716/{ticker}'):
        os.makedirs(f'results_ETF_0716/{ticker}')
    
    data.to_csv(f'results_ETF_0716/{ticker}/{ticker}_stock_data.csv')
    
    df = pd.read_csv(f'results_ETF_0716/{ticker}/{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['Return'] = df['Close'].pct_change()

    # Calculate technical indicators
    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['Upper_BB'], df['Middle_BB'], df['Lower_BB'] = talib.BBANDS(df['Close'], timeperiod=20)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

    # Plot technical indicators
    fig, ax = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

    df[['Close', 'MA10', 'MA20', 'Upper_BB', 'Middle_BB', 'Lower_BB']].plot(ax=ax[0], title=f'{ticker} - Stock Prices and Indicators')
    ax[0].set_ylabel('Price')
    
    df[['RSI']].plot(ax=ax[1], title='RSI')
    ax[1].set_ylabel('RSI')
    
    df[['MACD', 'MACD_signal']].plot(ax=ax[2], title='MACD')
    ax[2].fill_between(df.index, df['MACD_hist'], 0, color='gray', alpha=0.3)
    ax[2].set_ylabel('MACD')

    plt.tight_layout()
    plt.savefig(f'results_ETF_0716/{ticker}/{ticker}_indicators.png')
    plt.close()

    # Plot institutional trading
    institutional_trading_df = fetch_institutional_trading(ticker)
    fig, axs = plt.subplots(3, 1, figsize=(14, 18), sharex=True)

    sns.lineplot(data=institutional_trading_df, x='Date', y='Foreign_Investor_Buy', label='Foreign Investor Buy', ax=axs[0])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Foreign_Investor_Sell', label='Foreign Investor Sell', ax=axs[0])
    axs[0].set_title(f'{ticker} - Foreign Investors')
    
    sns.lineplot(data=institutional_trading_df, x='Date', y='Investment_Trust_Buy', label='Investment Trust Buy', ax=axs[1])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Investment_Trust_Sell', label='Investment Trust Sell', ax=axs[1])
    axs[1].set_title(f'{ticker} - Investment Trusts')
    
    sns.lineplot(data=institutional_trading_df, x='Date', y='Dealer_Buy', label='Dealer Buy', ax=axs[2])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Dealer_Sell', label='Dealer Sell', ax=axs[2])
    axs[2].set_title(f'{ticker} - Dealers')
    
    plt.tight_layout()
    plt.savefig(f'results_ETF_0716/{ticker}/{ticker}_institutional_trading.png')
    plt.close()

    # Prepare data for Prophet model
    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})

    # Train Prophet model
    model = Prophet()
    model.fit(prophet_df)

    # Predict next 30 days
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    # Plot forecast
    fig = model.plot(forecast)
    plt.title(f'{ticker} - Prophet Forecast')
    plt.tight_layout()
    plt.savefig(f'results_ETF_0716/{ticker}/{ticker}_prophet_forecast.png')
    plt.close()

    # Fetch and save news
    news_list = fetch_news(ticker)
    news_df = pd.DataFrame(news_list)
    news_df.to_csv(f'results_ETF_0716/{ticker}/{ticker}_news.csv', index=False)

    # 计算每日、每周、每月、半年和年收益率及胜率
    daily_return = df['Return'].mean() * 100
    weekly_return = df['Return'].resample('W').mean().mean() * 100
    one_month_return = df.loc[df.index[-1] - pd.DateOffset(months=1):, 'Return'].mean() * 100
    six_months_return = df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].mean() * 100
    one_year_return = df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].mean() * 100

    daily_win_rate = (df['Return'].apply(lambda x: x > 0).mean()) * 100
    monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()
    six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].empty else None
    one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].empty else None

    # 根据给定条件计算额外的胜率
    additional_win_rate = 0
    if df['Close'].iloc[-1] > df['Middle_BB'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['Close'].iloc[-1] > df['MA20'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['Close'].iloc[-1] > df['MA10'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['Close'].iloc[-1] > df['Upper_BB'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['RSI'].iloc[-1] > 60:
        additional_win_rate += 5
    if df['MACD'].iloc[-1] > 2:
        additional_win_rate += 5

    # 判断Prophet预测趋势
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    if forecast['yhat'].iloc[-1] > forecast['yhat'].iloc[-2]:
        additional_win_rate += 5

    # 判断三大法人是否连续三天买超
    institutional_trading_df['Net_Buy'] = institutional_trading_df['Foreign_Investor_Buy'] - institutional_trading_df['Foreign_Investor_Sell'] + \
                                          institutional_trading_df['Investment_Trust_Buy'] - institutional_trading_df['Investment_Trust_Sell'] + \
                                          institutional_trading_df['Dealer_Buy'] - institutional_trading_df['Dealer_Sell']
    if (institutional_trading_df['Net_Buy'].tail(3) > 0).all():
        additional_win_rate += 5

    total_win_rate = daily_win_rate + weekly_win_rate + monthly_win_rate + (six_months_win_rate if six_months_win_rate is not None else 0) + (one_year_win_rate if one_year_win_rate is not None else 0) + additional_win_rate

    return {
        'ticker': ticker,
        'daily_return': daily_return,
        'weekly_return': weekly_return,
        'one_month_return': one_month_return,
        'six_months_return': six_months_return,
        'one_year_return': one_year_return,
        'daily_win_rate': daily_win_rate,
        'monthly_win_rate': monthly_win_rate,
        'weekly_win_rate': weekly_win_rate,
        'six_months_win_rate': six_months_win_rate,
        'one_year_win_rate': one_year_win_rate,
        'additional_win_rate': additional_win_rate,
        'total_win_rate': total_win_rate,
        'df': df
    }

stock_tickers = get_stock_tickers()
results = []

for ticker in stock_tickers:
    result = analyze_stock(ticker)
    if result is not None:
        results.append(result)

# 创建DataFrame保存结果并导出为CSV
results_df = pd.DataFrame(results)
results_df = results_df[['ticker', 'daily_return', 'weekly_return', 'one_month_return', 'six_months_return', 'one_year_return', 
                         'daily_win_rate', 'weekly_win_rate', 'monthly_win_rate', 'six_months_win_rate', 'one_year_win_rate', 
                         'additional_win_rate', 'total_win_rate']]
results_df.to_csv("results_ETF_0716/weighted_win_rates.csv", index=False)


[*********************100%%**********************]  1 of 1 completed
15:00:10 - cmdstanpy - INFO - Chain [1] start processing
15:00:10 - cmdstanpy - INFO - Chain [1] done processing
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:73: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  ax.plot(m.history['ds'].dt.to_pydatetime(), m.history['y'], 'k.',
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_27589/255466

In [23]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
import talib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
from bs4 import BeautifulSoup

# 设置Seaborn样式
sns.set(style='whitegrid')

def get_stock_tickers():
    tickers = ["00929.TW", "00712.TW", "00637L.TW", "2884.TW", "00650L.TW", "2886.TW", 
               "2892.TW", "00893.TW", "00706L.TW", "2880.TW", "00830.TW", "00900.TW", 
               "0050.TW", "5880.TW", "00673R.TW", "00715L.TW", "00633L.TW", "2812.TW", 
               "00885.TW", "00895.TW", "00662.TW", "00888.TW", "00646.TW", "00903.TW", 
               "00683L.TW", "00642U.TW", "00770.TW", "00762.TW", "0051.TW", "00851.TW", 
               "006203.TW", "00660.TW"]
    return tickers

def fetch_news(ticker):
    url = f"https://tw.stock.yahoo.com/q/h?s={ticker}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_items = soup.find_all('li', class_='js-stream-content')

    news_list = []
    for item in news_items[:20]:  # Get top 20 news
        title = item.find('h3').text if item.find('h3') else 'No title'
        link = item.find('a')['href'] if item.find('a') else 'No link'
        news_list.append({'Title': title, 'Link': link})
    
    return news_list

def fetch_institutional_trading(ticker):
    # Simulated function to fetch institutional trading data
    # In real implementation, you would fetch this data from a reliable source
    dates = pd.date_range(end=pd.Timestamp('today'), periods=30)
    data = {
        'Date': dates,
        'Foreign_Investor_Buy': np.random.randint(100, 1000, size=30),
        'Foreign_Investor_Sell': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Buy': np.random.randint(100, 1000, size=30),
        'Investment_Trust_Sell': np.random.randint(100, 1000, size=30),
        'Dealer_Buy': np.random.randint(100, 1000, size=30),
        'Dealer_Sell': np.random.randint(100, 1000, size=30)
    }
    df = pd.DataFrame(data)
    return df
def analyze_stock(ticker):
    data = yf.download(ticker, start='2020-01-01', end='2024-07-17')
    if data.empty:
        return None
    
    if not os.path.exists(f'results_ETF_0717/{ticker}'):
        os.makedirs(f'results_ETF_0717/{ticker}')
    
    data.to_csv(f'results_ETF_0717/{ticker}/{ticker}_stock_data.csv')
    
    df = pd.read_csv(f'results_ETF_0717/{ticker}/{ticker}_stock_data.csv')
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['Return'] = df['Close'].pct_change()

    # Calculate technical indicators
    df['MA10'] = talib.SMA(df['Close'], timeperiod=10)
    df['MA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['Upper_BB'], df['Middle_BB'], df['Lower_BB'] = talib.BBANDS(df['Close'], timeperiod=20)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

    # Plot technical indicators
    fig, ax = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

    df[['Close', 'MA10', 'MA20', 'Upper_BB', 'Middle_BB', 'Lower_BB']].plot(ax=ax[0], title=f'{ticker} - Stock Prices and Indicators')
    ax[0].set_ylabel('Price')
    
    df[['RSI']].plot(ax=ax[1], title='RSI')
    ax[1].set_ylabel('RSI')
    
    df[['MACD', 'MACD_signal']].plot(ax=ax[2], title='MACD')
    ax[2].fill_between(df.index, df['MACD_hist'], 0, color='gray', alpha=0.3)
    ax[2].set_ylabel('MACD')

    plt.tight_layout()
    plt.savefig(f'results_ETF_0717/{ticker}/{ticker}_indicators.png')
    plt.close()

    # Plot institutional trading
    institutional_trading_df = fetch_institutional_trading(ticker)
    fig, axs = plt.subplots(3, 1, figsize=(14, 18), sharex=True)

    sns.lineplot(data=institutional_trading_df, x='Date', y='Foreign_Investor_Buy', label='Foreign Investor Buy', ax=axs[0])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Foreign_Investor_Sell', label='Foreign Investor Sell', ax=axs[0])
    axs[0].set_title(f'{ticker} - Foreign Investors')
    
    sns.lineplot(data=institutional_trading_df, x='Date', y='Investment_Trust_Buy', label='Investment Trust Buy', ax=axs[1])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Investment_Trust_Sell', label='Investment Trust Sell', ax=axs[1])
    axs[1].set_title(f'{ticker} - Investment Trusts')
    
    sns.lineplot(data=institutional_trading_df, x='Date', y='Dealer_Buy', label='Dealer Buy', ax=axs[2])
    sns.lineplot(data=institutional_trading_df, x='Date', y='Dealer_Sell', label='Dealer Sell', ax=axs[2])
    axs[2].set_title(f'{ticker} - Dealers')
    
    plt.tight_layout()
    plt.savefig(f'results_ETF_0717/{ticker}/{ticker}_institutional_trading.png')
    plt.close()

    # Prepare data for Prophet model
    prophet_df = df[['Close']].reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})

    # Train Prophet model
    model = Prophet()
    model.fit(prophet_df)

    # Predict next 30 days
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    # Plot forecast
    fig = model.plot(forecast)
    plt.title(f'{ticker} - Prophet Forecast')
    plt.tight_layout()
    plt.savefig(f'results_ETF_0717/{ticker}/{ticker}_prophet_forecast.png')
    plt.close()

    # Fetch and save news
    news_list = fetch_news(ticker)
    news_df = pd.DataFrame(news_list)
    news_df.to_csv(f'results_ETF_0717/{ticker}/{ticker}_news.csv', index=False)

    # 计算每日、每周、每月、半年和年收益率及胜率
    daily_return = df['Return'].mean() * 100
    weekly_return = df['Return'].resample('W').mean().mean() * 100
    one_month_return = df.loc[df.index[-1] - pd.DateOffset(months=1):, 'Return'].mean() * 100
    six_months_return = df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].mean() * 100
    one_year_return = df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].mean() * 100

    daily_win_rate = (df['Return'].apply(lambda x: x > 0).mean()) * 100
    monthly_win_rate = (df['Return'].resample('M').apply(lambda x: (x > 0).mean()) * 100).mean()
    weekly_win_rate = (df['Return'].resample('W').apply(lambda x: (x > 0).mean()) * 100).mean()
    six_months_win_rate = (df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(months=6):, 'Return'].empty else None
    one_year_win_rate = (df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].apply(lambda x: (x > 0)).mean() * 100) if not df.loc[df.index[-1] - pd.DateOffset(years=1):, 'Return'].empty else None

    # 根据给定条件计算额外的胜率
    additional_win_rate = 0
    if df['Close'].iloc[-1] > df['Middle_BB'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['Close'].iloc[-1] > df['MA20'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['Close'].iloc[-1] > df['MA10'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['Close'].iloc[-1] > df['Upper_BB'].iloc[-1] * 1.05:
        additional_win_rate += 5
    if df['RSI'].iloc[-1] > 60:
        additional_win_rate += 5
    if df['MACD'].iloc[-1] > 2:
        additional_win_rate += 5

    # 判断Prophet预测趋势
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    if forecast['yhat'].iloc[-1] > forecast['yhat'].iloc[-2]:
        additional_win_rate += 5

    # 判断三大法人是否连续三天买超
    institutional_trading_df['Net_Buy'] = institutional_trading_df['Foreign_Investor_Buy'] - institutional_trading_df['Foreign_Investor_Sell'] + \
                                          institutional_trading_df['Investment_Trust_Buy'] - institutional_trading_df['Investment_Trust_Sell'] + \
                                          institutional_trading_df['Dealer_Buy'] - institutional_trading_df['Dealer_Sell']
    if (institutional_trading_df['Net_Buy'].tail(3) > 0).all():
        additional_win_rate += 5

    total_win_rate = daily_win_rate + weekly_win_rate + monthly_win_rate + (six_months_win_rate if six_months_win_rate is not None else 0) + (one_year_win_rate if one_year_win_rate is not None else 0) + additional_win_rate

    return {
        'ticker': ticker,
        'daily_return': daily_return,
        'weekly_return': weekly_return,
        'one_month_return': one_month_return,
        'six_months_return': six_months_return,
        'one_year_return': one_year_return,
        'daily_win_rate': daily_win_rate,
        'monthly_win_rate': monthly_win_rate,
        'weekly_win_rate': weekly_win_rate,
        'six_months_win_rate': six_months_win_rate,
        'one_year_win_rate': one_year_win_rate,
        'additional_win_rate': additional_win_rate,
        'total_win_rate': total_win_rate,
        'df': df
    }

stock_tickers = get_stock_tickers()
results = []

for ticker in stock_tickers:
    result = analyze_stock(ticker)
    if result is not None:
        results.append(result)

# 创建DataFrame保存结果并导出为CSV
results_df = pd.DataFrame(results)
results_df = results_df[['ticker', 'daily_return', 'weekly_return', 'one_month_return', 'six_months_return', 'one_year_return', 
                         'daily_win_rate', 'weekly_win_rate', 'monthly_win_rate', 'six_months_win_rate', 'one_year_win_rate', 
                         'additional_win_rate', 'total_win_rate']]
results_df.to_csv("results_ETF_0717/weighted_win_rates.csv", index=False)


[*********************100%%**********************]  1 of 1 completed
15:06:35 - cmdstanpy - INFO - Chain [1] start processing
15:06:35 - cmdstanpy - INFO - Chain [1] done processing
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:73: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  ax.plot(m.history['ds'].dt.to_pydatetime(), m.history['y'], 'k.',
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_27589/290881

## 0717

In [30]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup
import logging
import asyncio
import aiohttp

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

[*********************100%%**********************]  1 of 1 completed
09:40:04 - cmdstanpy - INFO - Chain [1] start processing
09:40:04 - cmdstanpy - INFO - Chain [1] done processing
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()
/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/prophet/plot.py:73: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  ax.plot(m.history['ds'].dt.to_pydatetime(), m.history['y'], 'k.',
/var/folders/td/m175xz391hd4h1j11qwz0vyc0000gn/T/ipykernel_27589/402976

In [27]:
import os
import pandas as pd

# 文件路径
base_path_0717 = "/Users/tangjiahong/Dropbox/Pytorch/stock_env/stock_predict/results_ETF_0717"
base_path_0718 = "/Users/tangjiahong/Dropbox/Pytorch/stock_env/stock_predict/results_ETF_0718"

# 读取胜率文件
win_rates_0717 = pd.read_csv(os.path.join(base_path_0717, "weighted_win_rates.csv"))
win_rates_0718 = pd.read_csv(os.path.join(base_path_0718, "weighted_win_rates.csv"))

def get_close_price(base_path, ticker, date):
    stock_data_path = os.path.join(base_path, f"{ticker}", f"{ticker}_stock_data.csv")
    if os.path.exists(stock_data_path):
        stock_data = pd.read_csv(stock_data_path)
        stock_data.columns = stock_data.columns.str.strip()  # 去除列名中的空格
        stock_data['Date'] = pd.to_datetime(stock_data['Date'])
        stock_data.set_index('Date', inplace=True)
        if date in stock_data.index:
            return stock_data.loc[date, 'Close']
    return None

# 获取所有股票的收盘价和胜率变化
comparison_results = []

for ticker in win_rates_0717['ticker']:
    close_price_0717 = get_close_price(base_path_0717, ticker, '2024-07-17')
    close_price_0718 = get_close_price(base_path_0718, ticker, '2024-07-18')
    
    win_rate_0717 = win_rates_0717.loc[win_rates_0717['ticker'] == ticker, 'total_win_rate'].values[0]
    win_rate_0718 = win_rates_0718.loc[win_rates_0718['ticker'] == ticker, 'total_win_rate'].values[0]
    
    if close_price_0717 is not None and close_price_0718 is not None:
        price_change = close_price_0718 - close_price_0717
        win_rate_change = win_rate_0718 - win_rate_0717
        comparison_results.append({
            'ticker': ticker,
            'win_rate_change': round(win_rate_change, 2),
            'price_change': round(price_change, 2),
            'close_price_0717': round(close_price_0717, 2),
            'close_price_0718': round(close_price_0718, 2),
            'win_rate_0717': round(win_rate_0717, 2),
            'win_rate_0718': round(win_rate_0718, 2)
         })

# 创建 DataFrame 保存结果
comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_csv(os.path.join(base_path_0718, "comparison_results.csv"), index=False)

# 打印结果
print(comparison_df)


IndexError: index 0 is out of bounds for axis 0 with size 0